## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
import pyconll

In [2]:
# !mkdir datasets

In [3]:
# !wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
# !wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [4]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [5]:
for sent in full_train[:1]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT



In [6]:
for sent in full_train[:5]:
    sentense = [word.form for word in sent]
    print(' '.join(sentense), end='\n')

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .


In [7]:
def make_nltk_train(data_conllu):
    """
    Redo dconllu dataset for nltk taggers
    input:
        data_conllu: connllu data
    return:
        nltk_data: data (list(list(tupple(word,tag))))
    """
    nltk_data = []

    for sent in data_conllu:
        sentence = [(token.form, token.upos) for token in sent]
        nltk_data.append(sentence)
        
    return nltk_data
    

In [8]:
train = make_nltk_train(full_train)
test = make_nltk_train(full_test)


In [9]:
train[:2]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')]]

In [10]:
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger

In [11]:
unigram_tagger = UnigramTagger(train)
bigram_tagger = BigramTagger(train, backoff=unigram_tagger)
trigram_tagger = TrigramTagger(train, backoff=unigram_tagger)

In [12]:
for tagger in (unigram_tagger, bigram_tagger, trigram_tagger):
    print(f'train: {tagger.evaluate(train)})')
    print(f'test: {tagger.evaluate(test)})')

train: 0.9740030704763828)
test: 0.8772537323492737)
train: 0.9818043294175962)
test: 0.8829828463586425)
train: 0.9847095783717296)
test: 0.881769622215482)


In [13]:
from nltk.tag import TrigramTagger 

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = UnigramTagger(train)
tag = backoff_tagger(train,  
                     [BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test) 

0.882081353418933

In [14]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [15]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [18]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 8),  analyzer='char')

In [19]:
X_train = tfidf_vectorizer.fit_transform(train_tok)
X_test = tfidf_vectorizer.transform(test_tok)

In [20]:
lr = LogisticRegression(n_jobs=-1, penalty='l2')
lr.fit(X_train, train_enc_labels)

LogisticRegression(n_jobs=-1)

In [21]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8453391972500253

Не понимаю как на уроке получилось что общаая модель справилась лучше, по сути она имеет меньший биас чем модели по отдельности. видимо в моем случае биаса у моделей нету почти. Свой теггер вроде как справился лучше остальных. Tfidf, справился лучше чем CountVectorizer, возможно из за того что данные в CountVectorizere не нормированы, а может и из за самого подхода. В своем тегере попробовал взять lightgbm, но размеры данных слишком большие что бы строить деревья.

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [22]:
import corus

In [23]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)


процедуры обработки взять из вебинарного ноутбука

In [24]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\zald\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\zald\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [25]:
documents = list(records)


In [26]:
document = documents[1]

In [27]:
document.text

'Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, о

In [28]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document.text))) if hasattr(chunk, 'label') }

{('Thomas Hammarberg', 'PERSON'),
 ('Комиссар', 'PERSON'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('СЕ', 'ORGANIZATION'),
 ('Совета Европы', 'PERSON'),
 ('Хаммарберг', 'PERSON')}

Пожалуйста если есть возможность не пишите таких выражений они практически не читаемы. И да, NER от nltk такой себе =)

deeppavlov не установился ни сюда ни в venv. ему там много всего не нравится.

In [29]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)

In [33]:
from razdel import tokenize
import pandas as pd

In [32]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [34]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [35]:
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [36]:
df_words.head(3)

,word,tag
0,Россия,GEOPOLIT
1,рассчитывает,OUT
2,на,OUT


In [37]:
df_words.shape

(265471, 2)

In [38]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [39]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [40]:
train_x.apply(len).max(axis=0)

55

In [41]:
valid_x

89631           Президент
265315                  ,
41557                   о
32201                   ,
77144                   Ю
               ...       
236833                все
256401    припаркованного
226178           входящих
53421                   ,
9276            Президент
Name: word, Length: 66368, dtype: object

In [42]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [43]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [44]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [46]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [47]:
mmodel = modelNER()

In [48]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [50]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 225s 18ms/step - loss: 0.1390 - accuracy: 0.9565 - val_loss: 0.2395 - val_accuracy: 0.8859
Epoch 2/3
12444/12444 [==============================] - 213s 17ms/step - loss: 0.1270 - accuracy: 0.9583 - val_loss: 0.2276 - val_accuracy: 0.9346
Epoch 3/3
12444/12444 [==============================] - 266s 21ms/step - loss: 0.1224 - accuracy: 0.9589 - val_loss: 0.2400 - val_accuracy: 0.9342


Совершенно непонятно как это все работает, Для чего тут embeding слой, зачем тут maxpooling (просто для сокращения количствва данных?) Как построить другую форму сети по словам. Все это совершенно неочевидно. И по общей форме имеет мало отношения к курсу по изображениям. Для нас сетки это темный лес. Мы все проскакивали по верхам, без глобального разборы самого тензор флоу. В основном у нас были алгоритмы а не их реализация. И тут слои по сути конечно похожи но как точно они работают со словами непонятно. Если курс расчитан на то что вы за лецию показывает 10 библиотек по 10 мнут на каждую, я дома должен разобраться в их работе. То это как мне кажется чушь а не обучение.  Ямогу разобраться в том как подкрутить эту сетку, но у меня реально уйдет на это неделая, просто прочитать как все это работает по отдельности, и потыркаться на разных примеров. Учитывя то что сам Tensorflow сам по себе почти безразмерный. Чето накипело, простите.